In [1]:
import os
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"}
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [3]:
system_prompt = "You are assistant that analyzes contents of website and provide the short summary,\
focus on relevant text and respond in markdown "

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

def message(website):
    return [
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": user_prompt_for(website)}
    ]

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

openai = OpenAI()

In [5]:
def use_api(url='', online=False, model="llama3.2"):
    summary = ''
    if online:
        website = Website(url)
        response = openai.chat.completions.create(
            model = "gpt-4o-mini",
            messages = message(website)
        )
        summary = response.choices[0].message.content
        
    else:
        OLLAMA_API = "http://localhost:11434/api/chat"
        HEADERS = {"Content-Type": "application/json"}
        website = Website(url)
        payload = {
            "model": model,
            "messages": message(website),
            "stream": False
        }
        response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
        summary = response.json()['message']['content']

    return display(Markdown(summary))

In [6]:
use_api(url="https://edwarddonner.com", online=False)

**Summary of the Website**
=====================================

### Introduction

The website is owned by Edward Donner, a co-founder and CTO of Nebula.io, a company applying AI to help people discover their potential. The website also features an arena called "Outsmart" where LLMs (Large Language Models) compete against each other in a battle of diplomacy and deviousness.

### About the Owner

Edward Donner is a passionate writer, DJ, and amateur electronic music producer who enjoys experimenting with LLMs. He has co-founded Nebula.io, which uses AI to source, understand, engage, and manage talent.

### News and Announcements

* **December 21, 2024**: Announcement for the "LLM Workshop – Hands-on with Agents" event
* **November 13, 2024**: Welcome message for SuperDataScientists, a community of data scientists and AI enthusiasts
* **October 16, 2024**: Announcement for resources related to "Mastering AI and LLM Engineering"
* **January 23, 2025**: Upcoming event "LLM Workshop – Hands-on with Agents"

In [7]:
use_api(url="https://edwarddonner.com", online=True)

# Summary of Edward Donner's Website

Edward Donner's website serves as a personal platform where he shares his passion for coding, LLMs (Large Language Models), and his interests in DJing and music production. He is the co-founder and CTO of Nebula.io, focusing on utilizing AI for talent discovery and management. The site features sections like "Outsmart," which discusses LLM competitions, as well as his professional background, including a previous AI startup that was acquired in 2021.

## Recent Announcements
- **January 23, 2025**: Introduction of LLM Workshop focusing on hands-on experiences with agents; resources available.
- **December 21, 2024**: A welcome note for "SuperDataScientists."
- **November 13, 2024**: Release of resources for mastering AI and LLM engineering.
- **October 16, 2024**: Resources provided for transitioning from Software Engineer to AI Data Scientist. 

For further engagement, visitors can connect with him via LinkedIn, Twitter, and Facebook, or subscribe to his newsletter.